In [3]:
import pandas as pd
import patsy
import numpy as np

In [4]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca
import os; os.chdir('../')
import warnings;warnings.simplefilter('ignore')
from scripts import datasources, models, variables
from choicemodels import MultinomialLogit
from choicemodels import mnl
from choicemodels.tools import MergedChoiceTable

In [5]:
orca.run(['initialize_network_small', 'initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


In [4]:
persons = orca.get_table('persons').to_frame()
households = orca.get_table('households').to_frame()
buildings = orca.get_table('buildings').to_frame()
parcels = orca.get_table('parcels').to_frame()
jobs = orca.get_table('jobs').to_frame()
interaction_terms = pd.read_csv('./data/WLCM_interaction_terms.csv', index_col=['zone_id_home', 'zone_id_work'])
walk_net_vars = pd.read_csv('./data/walk_net_vars.csv', index_col='osmid')
drive_net_vars = pd.read_csv('./data/drive_net_vars.csv', index_col='osmid')

In [5]:
persons.columns

Index(['member_id', 'age', 'primary_commute_mode', 'relate', 'edu', 'sex',
       'hours', 'hispanic', 'earning', 'race_id', 'student', 'work_at_home',
       'worker', 'household_id', 'node_id_small', 'node_id_walk'],
      dtype='object')

In [6]:
households.columns

Index(['serialno', 'persons', 'building_type', 'cars', 'income',
       'race_of_head', 'hispanic_head', 'age_of_head', 'workers', 'state',
       'county', 'tract', 'block_group', 'children', 'tenure', 'recent_mover',
       'block_group_id', 'single_family', 'unit_id', 'building_id',
       'node_id_small', 'node_id_walk'],
      dtype='object')

In [7]:
commuters = persons[(persons['worker'] == 1) & (persons['work_at_home'] == 0)]

In [8]:
len(commuters)

3295341

In [9]:
obs = commuters.merge(households, left_on='household_id', right_index=True).merge(buildings, left_on='building_id', right_index=True).merge(
    parcels, left_on='parcel_id', right_index=True)[['zone_id']].rename(columns={'zone_id': 'zone_id_home'})
obs.index.name = 'obs_id'

In [10]:
len(obs)

3060996

In [11]:
obs.head()

,zone_id_home
obs_id,
0,557
3754,557
3755,557
22609,557
1,539


In [12]:
alts = jobs.merge(buildings, left_on='building_id', right_index=True).merge(
    parcels, left_on='parcel_id', right_index=True).merge(
    walk_net_vars, left_on='node_id_walk', right_index=True).merge(
    drive_net_vars, left_on='node_id_small', right_index=True).rename(columns={'zone_id': 'zone_id_work'})

In [13]:
alts = alts[['jobs_1500_walk', 'jobs_25000', 'zone_id_work']]

In [14]:
len(alts)

2578046

In [15]:
alts.head()

,jobs_1500_walk,jobs_25000,zone_id_work
job_id,,,
0,46.0,4987.0,297
1,46.0,4987.0,297
387,46.0,4987.0,297
388,46.0,4987.0,297
389,46.0,4987.0,297


In [16]:
%%time
mct = MergedChoiceTable(obs, alts,
                        sample_size=10, interaction_terms=interaction_terms)

CPU times: user 22.6 s, sys: 7.1 s, total: 29.7 s
Wall time: 29.8 s


In [17]:
mct_df = mct.to_frame()

In [6]:
mm.initialize(path='/home/max/projects/ual_model_workspace/fall-2018-models/configs/')

Registering model step 'WLCM'


In [8]:
m = mm.get_step('WLCM')

In [9]:
m.model_expression

'np.log1p(jobs_1500_walk) + np.log(jobs_25000) + np.log1p(tt_da) + tt_wTrnW - 1'

In [20]:
dm = patsy.dmatrix(m.model_expression, data=mct_df, return_type='dataframe')

In [21]:
probs = mnl.mnl_simulate(data = dm, coeff = m.fitted_parameters, 
                                 numalts = 10, returnprobs=True)

In [22]:
choice_positions = mnl.mnl_simulate(data = dm, coeff = m.fitted_parameters, 
                                            numalts = 10, returnprobs=False)

In [23]:
ids = mct_df.reset_index()[mct.alternative_id_col].tolist()

In [24]:
N = len(choice_positions)
J = len(ids) // N
ids_by_obs = np.reshape(ids, (N,J))
choices = [ids_by_obs[i][choice_positions[i]] for i in range(N)]

In [25]:
mct_df['probability'] = np.reshape(probs, (probs.size, 1))

In [26]:
mct_df

zone_id_home  jobs_1500_walk  jobs_25000  zone_id_work  \
obs_id  job_id                                                            
0       413751            557          2888.0    529017.0           583   
        1320102           557          4352.0    388068.0           875   
        1447013           557          8278.0    353118.0           309   
        2120253           557         16295.0    728987.0            40   
        2421446           557         78468.0    108870.0          1361   
        835143            557          1987.0    185191.0          1108   
        487342            557          2583.0     99222.0           718   
        1252816           557          9047.0    524827.0           340   
        678438            557         12474.0    281800.0           733   
        916320            557          1420.0    411762.0           768   
3754    1751724           557          4505.0    675060.0           147   
        430648            557           638.0    445496.0           653   
        1536123           557          3015.0    182620.0          1083   
        442118            557          4205.0     64132.0           700   
        2554759           557         65074.0     96705.0          1292   
        429930            557           141.0    442156.0           642   
        788673            557           843.0    726706.0           415   
        249077            557          4861.0    651294.0           541   
        2125771           557         36558.0    735959.0            32   
        230424            557          9365.0    615698.0           534   
3755    2208441           557          1103.0    585575.0           232   
        895329            557          1890.0    448918.0           773   
        1350778           557          6134.0    775656.0           931   
        2067971           557         22535.0    725495.0            41   
        1828141           557          3741.0    624227.0           244   
        1580869           557          6810.0    739008.0          1016   
        7458              557          4006.0    643268.0           370   
        1927217           557        140735.0    731967.0            23   
        851328            557          2535.0    204872.0          1161   
        1656859           557           533.0    719911.0          1014   
...                       ...             ...         ...           ...   
7060803 1136041          1403          2015.0    444531.0           857   
        1186504          1403          6994.0    549190.0           349   
        1350627          1403          6134.0    775656.0           931   
        1134020          1403          3436.0    206714.0          1094   
        1143396          1403          6072.0    434461.0           810   
        2323876          1403         17453.0    468308.0          1442   
        2578415          1403          5677.0    105273.0          1247   
        1172013          1403         12852.0    557719.0           349   
        608768           1403         12679.0    671492.0           608   
        1133850          1403          3436.0    206714.0          1094   
7060804 148048           1403          6913.0    714235.0           415   
        1241877          1403         11695.0    550494.0           340   
        570743           1403          3654.0    166729.0          1203   
        1929502          1403        132622.0    733024.0            23   
        1473534          1403         14314.0    332942.0           311   
        473293           1403            80.0     42568.0          1180   
        1657998          1403          8679.0    344009.0           281   
        362515           1403          1396.0    518005.0           682   
        2340017          1403         78468.0     63160.0          1399   
        268434           1403         10547.0    692767.0           435   
7060805 388808           1403          147

In [27]:
obs['choice'] = choices

In [28]:
obs.head()

,zone_id_home,choice
obs_id,,
0,557,413751
3754,557,230424
3755,557,895329
22609,557,102199
1,539,31870


In [29]:
obs[~obs.index.isin(commuters.index)]

,zone_id_home,choice
obs_id,,


In [30]:
merged = pd.merge(commuters, obs[['choice']], left_index=True, right_index=True, how='left').rename(columns={'choice': 'job_id'})

In [36]:
len(merged)

3295341

In [39]:
merged = pd.merge(persons, merged[['job_id']], left_index=True, right_index=True, how='left')

In [42]:
len(merged) == len(persons)

True

In [47]:
merged.to_csv('/home/max/projects/ual_model_workspace/fall-2018-models/data/persons_w_jobs.csv')